In [74]:
import pandas as pd

# 1. Carregar o arquivo CSV em um DataFrame
# Certifique-se de que o arquivo 'manga.csv' está no mesmo diretório
# do seu script Python, ou forneça o caminho completo para o arquivo.

df_manga = pd.read_csv('manga.csv')

# 2. Exibir as 5 primeiras linhas (o padrão do .head() é 5)
print(df_manga.head())

   manga_id                                            title   type  score  \
0         2                                          Berserk  manga   9.47   
1        13                                        One Piece  manga   9.22   
2      1706  JoJo no Kimyou na Bouken Part 7: Steel Ball Run  manga   9.30   
3      4632                                   Oyasumi Punpun  manga   9.02   
4        25                              Fullmetal Alchemist  manga   9.03   

   scored_by                status  volumes  chapters  start_date    end_date  \
0     319696  currently_publishing      NaN       NaN  1989-08-25         NaN   
1     355375  currently_publishing      NaN       NaN  1997-07-22         NaN   
2     151433              finished     24.0      96.0  2004-01-19  2011-04-19   
3     168459              finished     13.0     147.0  2007-03-15  2013-11-02   
4     153151              finished     27.0     116.0  2001-07-12  2010-09-11   

   ...           demographics  \
0  ...     

In [75]:
# Lista de colunas a serem eliminadas
colunas_a_remover = [
    'volumes',
    'chapters',
    'start_date',
    'end_date',
    'approved',
    'created_at_before',
    'updated_at',
    'real_start_date',
    'real_end_date',
    'synopsis',
    'background',
    'main_picture',
    'url',
    'title_english',
    'title_japanese',
    'serializations',
    'authors',
    'title_synonyms'
]

# A coluna 'sfw' será tratada na próxima etapa, onde as LINHAS com 'sfw=False' serão removidas.

In [76]:
# --- PASSO 1: Remover as colunas ---
print("Iniciando a remoção das colunas...")

# Usamos axis=1 para indicar que estamos removendo COLUNAS
# Usamos errors='ignore' caso alguma coluna da lista não exista no seu DataFrame
df_manga_limpo = df_manga.drop(
    columns=colunas_a_remover, 
    axis=1, 
    errors='ignore'
)

# --- PASSO 2: Remover as linhas onde 'sfw' é False ---
if 'sfw' in df_manga_limpo.columns:
    
    df_manga_limpo = df_manga_limpo[df_manga_limpo['sfw'] == True]   
    df_manga_limpo = df_manga_limpo.drop(columns=['sfw'], errors='ignore')

else:
    print("A coluna 'sfw' não está presente no DataFrame (possivelmente foi removida anteriormente ou não existe). Pulando a filtragem.")

# --- Visualização do Resultado ---
print("\n--- Resultado da Limpeza ---")
print(f"Tamanho do DataFrame Original: {len(df_manga)} linhas")
print(f"Tamanho do Novo DataFrame (df_manga_limpo): {len(df_manga_limpo)} linhas")

Iniciando a remoção das colunas...

--- Resultado da Limpeza ---
Tamanho do DataFrame Original: 64833 linhas
Tamanho do Novo DataFrame (df_manga_limpo): 45157 linhas


In [77]:
# Nome do novo arquivo CSV
novo_csv = 'manga_limpo.csv'

df_manga_limpo.to_csv(novo_csv, index=False)

print(f"Ele contém *{len(df_manga_limpo)} linhas e *{len(df_manga_limpo.columns)} colunas.")
print(df_manga_limpo.head())

Ele contém *45157 linhas e *11 colunas.
   manga_id                                            title   type  score  \
0         2                                          Berserk  manga   9.47   
1        13                                        One Piece  manga   9.22   
2      1706  JoJo no Kimyou na Bouken Part 7: Steel Ball Run  manga   9.30   
3      4632                                   Oyasumi Punpun  manga   9.02   
4        25                              Fullmetal Alchemist  manga   9.03   

   scored_by                status  members  favorites  \
0     319696  currently_publishing   643969     119470   
1     355375  currently_publishing   579557     111462   
2     151433              finished   248511      41713   
3     168459              finished   413897      49361   
4     153151              finished   284027      29634   

                                              genres  \
0  ['Action', 'Adventure', 'Award Winning', 'Dram...   
1                 ['Action', '

##

## Tranformação dos Dados 2º Etapa

In [79]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Supondo que 'df_manga_limpo' é o seu DataFrame atual
df_transformado = pd.read_csv('manga_limpo2.csv')

In [80]:
# Lista das colunas a serem transformadas
colunas_categoricas = ['genres', 'themes', 'demographics']

# Iterar sobre cada coluna e aplicar a codificação
for col in colunas_categoricas:
    if col in df_transformado.columns:

        # --- Passo 1: Limpeza e Preparação da String ---
        # Garantir que a coluna é do tipo string e lidar com valores NaN restantes 
        # (Embora você tenha dito que não há, é bom ser robusto)
        df_transformado[col] = df_transformado[col].astype(str).replace('nan', '', regex=False)
        
        # Limpar strings (remover colchetes, aspas e outros caracteres)
        df_transformado[col] = df_transformado[col].str.replace(r"[\[\]'\"]", '', regex=True)
        
        # --- Passo 2: Aplicação do One-Hot Encoding ---
        
        # O método str.get_dummies() divide a string pelo separador (', ') 
        # e cria colunas binárias para cada termo.
        # Adiciona o prefixo para identificar de qual coluna original a nova veio.
        dummies = df_transformado[col].str.get_dummies(sep=', ').add_prefix(f'{col}_')
        
        # Concatenar o novo DataFrame de dummies ao DataFrame principal
        df_transformado = pd.concat([df_transformado, dummies], axis=1)
        
        # --- Passo 3: Remover a Coluna Original ---
        # A coluna original ('genres', 'themes', 'demographics') não é mais necessária
        df_transformado = df_transformado.drop(columns=[col])
        
    else:
        print(f"⚠️ Aviso: A coluna '{col}' não foi encontrada no DataFrame. Pulando.")


# --- Visualização do Resultado ---
colunas_novas = [c for c in df_transformado.columns if any(c.startswith(prefix) for prefix in ['genres_', 'themes_', 'demographics_'])]
print(df_transformado[['title'] + colunas_novas].head())

print(f"\nNúmero de colunas antes: {len(df_manga_limpo.columns)}")
print(f"Número de colunas depois: {len(df_transformado.columns)}")

⚠️ Aviso: A coluna 'genres' não foi encontrada no DataFrame. Pulando.
⚠️ Aviso: A coluna 'themes' não foi encontrada no DataFrame. Pulando.
⚠️ Aviso: A coluna 'demographics' não foi encontrada no DataFrame. Pulando.
                                             title  genres_Action  \
0                                          Berserk              1   
1                                        One Piece              1   
2  JoJo no Kimyou na Bouken Part 7: Steel Ball Run              1   
3                                   Oyasumi Punpun              0   
4                              Fullmetal Alchemist              1   

   genres_Adventure  genres_Avant Garde  genres_Award Winning  \
0                 1                   0                     1   
1                 1                   0                     0   
2                 1                   0                     0   
3                 0                   0                     0   
4                 1                   0     

In [81]:

#df_transformado = df_transformado[df_transformado['genres_Ecchi'] == 0]   
#df_transformado = df_transformado.drop(columns=['genres_Ecchi'], errors='ignore')
df_transformado = df_transformado[df_transformado['members'] >= 1000]
df_transformado = df_transformado[df_transformado['score'] >= 7]

In [82]:
# --- 1. Carregar o Arquivo Mais Limpo ---
df = pd.read_csv('manga_limpo2.csv')

# --- 2. Preparação dos Dados Numéricos ---

# Colunas numéricas que definem Qualidade e Popularidade/Engajamento.
cols_metricas_originais = ['score', 'scored_by', 'members', 'favorites']

# 🚨 Garantir que não há NaN antes das transformações
df[cols_metricas_originais] = df[cols_metricas_originais].fillna(0)

# Criar as colunas logarítmicas (para reduzir a distorção dos outliers)
df['log_scored_by'] = np.log1p(df['scored_by'])
df['log_members'] = np.log1p(df['members'])
df['log_favorites'] = np.log1p(df['favorites'])

# Colunas que serão padronizadas (Min-Max) - Incluindo o 'score' e o novo 'log_scored_by'
cols_a_padronizar = ['score', 'log_scored_by', 'log_members', 'log_favorites']

# --- 3. Aplicar Padronização Min-Max ---

# Inicializar o MinMaxScaler
scaler = MinMaxScaler()

# Ajustar e transformar as 4 colunas selecionadas
dados_padronizados = scaler.fit_transform(df[cols_a_padronizar])

# Converter o array de volta para um DataFrame
df_padronizado = pd.DataFrame(
    dados_padronizados,
    columns=[f'{col}' for col in cols_a_padronizar]
)

# --- 4. Unir os Dados Padronizados ao DataFrame Original ---

# Colunas originais a serem removidas para evitar redundância no DataFrame final
cols_a_dropar = ['members', 'favorites', 'scored_by'] + [col for col in df.columns if col.startswith('log_')]
df_original_para_concat = df.drop(columns=cols_a_dropar)

# Unir o DataFrame limpo e com categorias ao DataFrame padronizado
df_final = pd.concat([
    df_original_para_concat.reset_index(drop=True), 
    df_padronizado
], axis=1)

In [83]:
novo_csv = 'manga_minmax.csv'

#df_transformado.to_csv(novo_csv, index=False)
df_final.to_csv(novo_csv, index=False)

In [ ]:
# --- 1. Identificar e Somar Todas as Features de Conteúdo ---
df = df_final
# Identificar todas as colunas de conteúdo (Gêneros, Temas, Demografias)
cols_conteudo = [col for col in df.columns if col.startswith(('genres_', 'themes_', 'demographics_'))]

# Criar um Series com a frequência total de cada categoria
frequencia_categorias = df[cols_conteudo].sum().sort_values(ascending=False)

# --- 2. Selecionar o Top 9 e as Colunas a serem Agrupadas ---

N_TOP = 9

# Selecionar os nomes das 9 colunas mais frequentes
top_9_cols = frequencia_categorias.head(N_TOP).index.tolist()

# Selecionar os nomes das colunas que não estão no Top 9
cols_to_group = frequencia_categorias.index.difference(top_9_cols).tolist()

# --- 3. Criar a Coluna 'Content_Others' e Remover as Originais ---

# Calcular a soma das colunas menos populares (as que serão agrupadas)
df['Content_Others'] = df[cols_to_group].sum(axis=1)

# As colunas originais (top_9 + o que foi agrupado) serão removidas
cols_a_remover = cols_to_group

# Remover as colunas de baixa frequência do DataFrame
df_reduzido = df.drop(columns=cols_a_remover)

# --- 4. Exibir o Resultado e as Novas Colunas ---

print(f"🎉 Redução Dimensional Concluída! Total de categorias originais: {len(cols_conteudo)}")
print(f"   - {len(cols_to_group)} categorias de baixa frequência foram agrupadas.")
print(f"   - O DataFrame final agora possui {len(df_reduzido.columns)} colunas, incluindo as 9 Top features e 'Content_Others'.")

print("\n### Top 9 Categorias Mantidas ###")
for col in top_9_cols:
    print(f"  - {col}")

print(f"  - Content_Others (Agrupamento)")

# --- 5. Exportar o Novo DataFrame Reduzido ---
nome_novo_arquivo = 'manga_features_reduzidas.csv'

try:
    df_reduzido.to_csv(nome_novo_arquivo, index=False)
    print(f"\n💾 Novo arquivo com features reduzidas salvo como: **{nome_novo_arquivo}**")
except Exception as e:
    print(f"❌ Erro ao salvar o arquivo: {e}")

🎉 Redução Dimensional Concluída! Total de categorias originais: 75
   - 66 categorias de baixa frequência foram agrupadas.
   - O DataFrame final agora possui 19 colunas, incluindo as 9 Top features e 'Content_Others'.

### Top 9 Categorias Mantidas ###
  - genres_Romance
  - genres_Comedy
  - genres_Drama
  - genres_Fantasy
  - themes_School
  - demographics_Shoujo
  - genres_Action
  - demographics_Shounen
  - demographics_Seinen
  - Content_Others (Agrupamento)

💾 Novo arquivo com features reduzidas salvo como: **manga_features_reduzidas.csv**
